# ac(adding calculator) 구문 분석기 

All rights reserved, 2021, By Youn-Sik Hong. 수업 목적으로만 활용 가능.

- 강의 노트 **ch2-1, ch2-2** 내용을 충분히 들여다 보기 바랍니다.
- 구체적인 코드를 이해할 필요는 없습니다. 나무를 보려는 게 아니라 숲을 보려고 하기 때문이죠.
    - Not see the forest for the trees!
- 컴파일러 전반부(front-end)의 구현 과정에 대한 감(sense)을 잡기위한 것입니다.
    - nltk에서 제공하는 라이브러리를 사용하면 훨씬 간단하게 구현할 수 있습니다.
        - 그런데, 그건 구현이 아니라 라이브러리 사용 방법을 익히는 것뿐이죠.
        - 이 과목을 수강하지 않더라도 누구나 그 정도는 할 수 있습니다.
- 여러분은 nltk에서 제공하는 라이브러리 정도는 직접 만들어, github에서 제공하고 싶지 않나요?

- 코드를 단순화하려면 import문을 사용하여 어휘분석기를 포함하면 됩니다.
    - 어휘분석기 코드를 다시 한 번 들여다 보는 것이 전체적인 이해에 도움이 될 것 같아 다시 포함했습니다. 
    - 어휘분석기 코드에서 테스트용으로 사용한 코드는 제외했습니다.

In [1]:
class Token:
    def __init__(self, typ, val):
        self.type = typ
        self.value = val  

In [2]:
def peek(i):
    return istream[i] 

In [3]:
def advance(i, lim):
    i += 1
    if (i < lim):
        s = istream[i]  
    else: 
        s = None
    return i, s

In [4]:
def ScanDigit(idx):
    val = ""  #빈 문자열(str), 초기화.     
    s = peek(idx)
    limit = len(istream)
    
    while s.isdigit():
        val += s
        idx, s = advance(idx, limit)
        if s == None:
            return idx, Token('INUM', val)
    
    if (idx < limit and s != '.'):
        type = 'INUM'
    else:    
        type = 'FNUM'
        val += s     
        idx, s = advance(idx, limit)     
        if s == None:
            return idx, Token(type, val)            
        
        while s.isdigit():
            val += s  
            idx, s = advance(idx, limit)                     
            if s == None:
                break   
                    
    return idx, Token(type, val)   

In [5]:
def representativeChar(c):
    if c.isalpha():
        if c not in ['f', 'i', 'p']:
            return True
    return False

In [6]:
def Scanner(idx):
    limit = len(istream)  
    val = ""    
    ans = Token('EOF', None)
    
    if idx >= limit-1:
        return idx, ans
    
    s = peek(idx)    
    while s == ' ':
        idx, s = advance(idx, limit)     
    
    if s != None:
        if s.isdigit():
            idx, ans = ScanDigit(idx)
        else:
            if representativeChar(s):
                ans = Token('ID', s)            
            elif s == 'f':
                ans = Token('FLTDCL', None)
            elif s == 'i':
                ans = Token('INTDCL', None)
            elif s == 'p':
                ans = Token('PRINT', None)
            elif s == '=':
                ans = Token('ASSIGN', None)
            elif s == '+':
                ans = Token('PLUS', None)
            elif s == '-':
                ans = Token('MINUS', None)   
            else:
                ans = Token('ERROR', s)
    
    return idx, ans

In [7]:
istream = "f b   i a   a = 5   b = a + 3.2   p b"
index, tok = Scanner(2)
limit = len(istream)  

idx, tok = Scanner(index) 
print(idx, tok.type, tok.value)

2 ID b


- 아래 코드는 **Recursive Descent Parsing** 알고리즘으로 구현하였습니다.
    - 파싱 알고리즘 중에서는 성능이 가장 나쁘지만, 파서를 빨리 구현할 수 있다는 장점이 있습니다.
        - 생성규칙의 왼쪽(*lhs*)에 있는 모든 *Nonterminal* 에 대해 개별 함수를 만듭니다.
            - 생성규칙의 오른쪽(rhs)에 *Nonterminal* 이 있으면 해당 함수를 호출하면 됩니다.
        - 생성규칙에 *terminal*이 있으면 **match()** 함수를 사용해 입력에 그런 토큰이 있는지 비교하면 됩니다.
    - 파이썬 *class* 로 구현했으며, 강의노트(**ch2-2.pdf**) 를 잘 보면서 확인하기 바랍니다.
        - 조금 고민되는 부분은 *Nonterminal* 함수에서 사용한 *if* 문이 왜 이런 저런 type을 조사했는지 일겁니다.
        - 이 부분은 컴파일러 이론을 조금 더 배우면 쉽게 알 수 있습니다.
    - 파이썬은 아무 문장도 없으면 에러가 발생합니다.
        - 대신 *pass* 라는 문장이 있습니다. 
    - 각 함수의 print 문을 주석 처리하였습니다. 
        - print 문 주석을 없애고 실행시켜 보면 코드 이해에 도움이 될 겁니다.

In [8]:
def match(idx,token,terminal):     
    if token.type != terminal:
        print("syntax error", terminal, token.type)
        exit()
    else:
        if idx-1 < len(istream):             
            idx += 1
            #print('before match', .idx, .tok.type)            
            idx,token = Scanner(idx) 
            #print('after match',.idx, .tok.type)
        else:
            exit()    
        return idx,token

In [9]:
# match(idx,tok,'FLTDCL')

In [10]:
class Parser():
    def __init__(self, tok, idx): # 생성자(constructor)
        self.tok = tok
        self.idx = idx
        
    def Prog(self): # member method
        if (self.tok.type == 'FLTDCL' or self.tok.type == 'INTDCL' or self.tok.type == 'ID' 
            or self.tok.type == 'PRINT' or self.tok.type == 'EOF'):
            self.Dcls()
            self.Stmts()
            self.match('EOF')
        else:
            print("expected floatdcl, intdcl, id, print, or eof" , tok.type)            

    def Dcls(self):
        #print('Dcls', self.idx, self.tok.type, self.tok.value)        
        
        if (self.tok.type == 'FLTDCL' or self.tok.type == 'INTDCL'):
            self.Dcl()
            self.Dcls()
            
        elif (self.tok.type == 'ID' or self.tok.type == 'PRINT' or self.tok.type == 'EOF'):
            #Do nothing for lambda-production
            pass  
        else:
            print("expected floatdcl, intdcl, id, print, or eof")
            
    def Dcl(self):
        #print('Dcl', self.idx, self.tok.type, self.tok.value)        
        
        if self.tok.type == 'FLTDCL':
            self.match('FLTDCL')
            self.match('ID')           
        elif self.tok.type == 'INTDCL':
            self.match('INTDCL')
            self.match('ID')     
        else:
            print("expected float or int declaration")

    def Stmts(self):
        #print('Stmts', self.idx, self.tok.type, self.tok.value)        
        
        if (self.tok.type == 'ID' or self.tok.type == 'PRINT'):
            self.Stmt()
            self.Stmts()
        elif self.tok.type == 'EOF':
            #Do nothing for lambda-production
            pass                                     
        else:
            print("expected id, print, or eof")          
            
    def Stmt(self):
        #print('Stmt', self.idx, self.tok.type, self.tok.value)        
        
        if self.tok.type == 'ID':
            self.match('ID')
            self.match('ASSIGN')
            self.Val()
            self.Expr()
        elif self.tok.type == 'PRINT':
            self.match('PRINT')
            self.match('ID')
        else:
            print("expected id or print")               

    def Expr(self):
        #print('Expr', self.idx, self.tok.type, self.tok.value)        
        
        if self.tok.type == 'PLUS':
            self.match('PLUS')
            self.Val()
            self.Expr()
        elif self.tok.type == 'MINUS':
            self.match('MINUS')
            self.Val()
            self.Expr()
        elif (self.tok.type == 'ID' or self.tok.type == 'PRINT' or self.tok.type == 'EOF'): 
            #Do nothing for lambda-production  
            pass
        else:
            print("expected plus, minus, id, print, or eof")              

    def Val(self):
        #print('Val', self.idx, self.tok.type, self.tok.value)        
        if self.tok.type == 'ID':
            self.match('ID')
        elif self.tok.type == 'INUM':
            self.match('INUM') 
        elif self.tok.type == 'FNUM':
            self.match('FNUM') 
        else:            
            print("expected id, inum, or fnum")            
            
    def match(self, t):     
        print('match', self.idx, self.tok.type, self.tok.value)        
        
        if self.tok.type != t:
            print("syntax error", t, self.tok.type)
            exit()
        else:
            if self.idx-1 < len(istream):             
                self.idx += 1
                #print('before match', self.idx, self.tok.type)            
                self.idx, self.tok = Scanner(self.idx) 
                #print('after match',self.idx, self.tok.type)
            else:
                exit()            

- 이제 구문분석기가 입력 문장(*istream*)을 문법에 맞게 분석하는지 확인해 보겠습니다.
    - 구문 분석을 위해 먼저 토큰 한 개(*tok*)를 읽어 옵니다.
        - **Parser** 클래스에서 **match()** 메소드를 호출할 때마다 다음 *token* 을 가져 옵니다.
    - **Parser** 객체 *p* 의 멤버 메소드에서 **Prog()** 메소드를 호출한 점에 주목해 주세요.

In [ ]:
p = Parser(tok,0)

In [11]:
istream = "f b   i a   a = 5   b = a + 3.2   p b"
index, tok = Scanner(0)
limit = len(istream)  

idx, tok = Scanner(index) 
print(idx, tok.type, tok.value)
p = Parser(tok, 0)
p.Prog()

0 FLTDCL None
match 0 FLTDCL None
match 2 ID b
match 6 INTDCL None
match 8 ID a
match 12 ID a
match 14 ASSIGN None
match 17 INUM 5
match 20 ID b
match 22 ASSIGN None
match 24 ID a
match 26 PLUS None
match 31 FNUM 3.2
match 34 PRINT None
match 36 ID b
match 37 EOF None
